In [1]:
from helper.neural_net import *
from models.autoencoder import *

%load_ext autoreload
%autoreload 2

In [ ]:
run_experiment('autoencoder', 'mse', 'adam', image_dir, gt_dir, num_epochs=1, autoencoder=True)

Loaded 100 images
All images have the same size !
Image size = 400, 400
Things will go much quicker with a GPU
Starting training
tensor([[[[0.2118, 0.2000, 0.2000,  ..., 0.3373, 0.3059, 0.2667],
          [0.1843, 0.1961, 0.2118,  ..., 0.3255, 0.3098, 0.3137],
          [0.1843, 0.2157, 0.2275,  ..., 0.3333, 0.3412, 0.3137],
          ...,
          [0.5176, 0.4275, 0.4784,  ..., 0.5176, 0.4745, 0.4157],
          [0.5294, 0.4235, 0.3647,  ..., 0.5098, 0.4706, 0.4039],
          [0.5020, 0.4078, 0.3059,  ..., 0.5098, 0.5098, 0.4078]],

         [[0.2549, 0.2471, 0.2471,  ..., 0.3216, 0.2863, 0.2471],
          [0.2314, 0.2431, 0.2549,  ..., 0.3059, 0.2902, 0.2941],
          [0.2314, 0.2588, 0.2784,  ..., 0.3137, 0.3255, 0.2941],
          ...,
          [0.4902, 0.4000, 0.4549,  ..., 0.5098, 0.4667, 0.4078],
          [0.5059, 0.4000, 0.3373,  ..., 0.5020, 0.4667, 0.3922],
          [0.4745, 0.3882, 0.2824,  ..., 0.4980, 0.5020, 0.3961]],

         [[0.1647, 0.1569, 0.1529,  ..., 0.29